In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [3]:
from LCQMC.LCQMC_test_evaluate import Metric

In [4]:
%cd /content/drive/My Drive/LCQMC/Sentence-BERT/

/content/drive/My Drive/LCQMC/Sentence-BERT


In [5]:
! pip3 install sentence_transformers

     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 18.3MB/s 
     |████████████████████████████████| 3.0MB 32.0MB/s 
     |████████████████████████████████| 1.1MB 57.4MB/s 
     |████████████████████████████████| 890kB 55.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=a7c700a9f855df22630ac74a95e6da7bbe0d45a200bee42148a296f5ae1d1733
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2f2e1596f8abb21224d07812a23435cdcdd87ed52f65914495f95e255c4ad3e4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [6]:
from sentence_transformers import SentenceTransformer, models, util, SentencesDataset, InputExample, losses
from torch import nn
from torch.utils.data import DataLoader
import transformers
from LabelAccuracyEvaluator import LabelAccuracyEvaluator
import pandas as pd
import numpy as np
import torch
import os

def test_evaluate(test_path, model_save_path):
  test=pd.read_csv(test_path, sep='\t', names=['s1','s2','similarity'])
  test_examples = []
  for i in range(len(test)):
    test_examples.append(InputExample(texts=[test.s1[i],test.s2[i]], label=int(test.similarity[i])))

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model = SentenceTransformer(model_save_path)
  model = model.to(device)
  test_dataset = SentencesDataset(test_examples, model)
  test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=100)
  train_loss = torch.load(os.path.join(model_save_path,'3_Softmax/pytorch_model.bin'))
  evaluator = LabelAccuracyEvaluator(test_dataloader, softmax_model = train_loss)
  model.evaluate(evaluator, output_path = model_save_path)

if __name__ == '__main__':
  test_path = '/content/drive/My Drive/LCQMC/data/LCQMC_test.tsv'
  model_save_path = '/content/drive/My Drive/LCQMC/Sentence-BERT/output_little_sbert_cnn'
  test_evaluate(test_path, model_save_path)

Evaluating: 100%|██████████| 125/125 [00:15<00:00,  8.31it/s]


In [6]:
test_join, test_wrong_classify, test_true_classify = Metric('/content/drive/My Drive/LCQMC/output/Roberta/test_prediction.csv')

Accuracy: 0.87104
Precision: 0.8803924193154709
Recall: 0.87104
F1: 0.8702424373557535
classification_report:

              precision    recall  f1-score   support

     class_0     0.9400    0.7926    0.8601      6250
     class_1     0.8207    0.9494    0.8804      6250

    accuracy                         0.8710     12500
   macro avg     0.8804    0.8710    0.8702     12500
weighted avg     0.8804    0.8710    0.8702     12500



In [ ]:
test_wrong_classify.head(100)

,s1,s2,label,prob_0,prob_1,prediction
12,心各有所属是什么意思？,心有所属是什么意思?,0,-1.105229,0.434929,1
14,世界杯哪位球员进球最多,世界杯单界进球最多是哪位球员,0,-1.254105,1.141075,1
21,淘宝上怎么用信用卡分期付款,淘宝怎么分期付款，没有信用卡,0,-0.778508,1.042356,1
23,《校花的贴身高手》中的林逸,校花贴身高手,1,-0.073772,-0.349846,0
25,这姑娘漂亮不,我姑娘漂亮吧,0,-0.863732,1.671991,1
...,...,...,...,...,...,...
628,晚安，用韩语怎么说？,晚安的韩语怎么写？,0,-0.772601,0.964531,1
643,天津海运职业学院,天津海运职业学院在哪个区,0,-0.408320,0.933107,1
644,爱江山更爱美人,爱江山更爱美人这样的歌,0,-1.102258,1.200551,1
648,为什么我的声音不好听？,为什么录出来的声音不好听,0,-0.405490,0.486841,1


In [ ]:
test_wrong_classify.label.value_counts()

0    1672
1     224
Name: label, dtype: int64

In [ ]:
test_wrong_classify.head(100)

,s1,s2,label,prob_0,prob_1,prediction
14,世界杯哪位球员进球最多,世界杯单界进球最多是哪位球员,0,-0.176001,0.429758,1
21,淘宝上怎么用信用卡分期付款,淘宝怎么分期付款，没有信用卡,0,-0.467642,0.573164,1
25,这姑娘漂亮不,我姑娘漂亮吧,0,-0.488526,0.194062,1
34,什么牌子的精油皂好,什么牌子的精油好？,0,-0.292229,0.835557,1
35,刚出生的小野鸡怎么养,刚抓来的野鸡怎么养殖,0,-1.249436,0.414353,1
...,...,...,...,...,...,...
673,今天天气不错，你的心情好吗,你今天心情不好吗.PDF,0,-2.182365,1.559173,1
674,大家都笑你打一中国城市名),猜一猜打中国城市,1,2.384317,-1.256046,0
678,不等式组的解集为()。,不等式的解集是,0,-1.803813,0.585797,1
686,神庙逃亡的兑换码是什么？,"神庙逃亡2,的兑换码是什么",0,-1.077119,2.679847,1


In [ ]:
test_wrong_classify.label.value_counts()

0    1540
1     292
Name: label, dtype: int64